## 2.  핫플지수 확정



In [1]:
import polars as pl
import numpy as np

print("✅ 1️⃣ Parquet 스캔 시작")
lazy_df = pl.scan_parquet('combined_parquet/**/*.parquet', glob=True)
print("   → LazyFrame 생성 완료")


✅ 1️⃣ Parquet 스캔 시작
   → LazyFrame 생성 완료


In [2]:
import polars as pl

# ———————— 풀 테이블 출력 설정 ————————
pl.Config.set_tbl_rows(100)         # 최대 100행까지 줄임표 없이 표시
pl.Config.set_tbl_cols(100)        # 최대 100열까지 표시
pl.Config.set_tbl_formatting("UTF8_FULL")  # 테두리 포함, 생략 없이 전체 보여주기


polars.config.Config

In [8]:

import polars as pl


# 1️⃣ filtered 정의: 2024년··주요 목적 필터 + 날짜 파싱
filtered = (
    lazy_df
      # ─ 날짜 범위: 2024-01-01 ~ 2024-12-31
      .filter((pl.col("ETL_YMD") >= "20240101") & (pl.col("ETL_YMD") <= "20241231"))
      
      # ─ 시간 범위: 10시부터 23시까지
      .filter((pl.col("TIME_CD").cast(pl.Int64) >= 18) 
            & (pl.col("TIME_CD").cast(pl.Int64) <= 23))
      
      # ─ 목적 필터: 관심 목적군만
      .filter(pl.col("MOVE_PURPOSE").is_in(
          ["쇼핑","관광","출근","귀가","병원","기타"]
      ))


      # ─ ETL_YMD 문자열을 Date 타입으로 바꿔서
      .with_columns(
          pl.col("ETL_YMD")
            .str.strptime(pl.Date, format="%Y%m%d")
            .alias("ETL_DATE")
      )
          # ─ 주말(weekday 5,6)만
      .filter(pl.col("ETL_DATE").dt.weekday().is_in([5,6]))
   
)

# 이제 `filtered`에 “원하는 기간·시간·목적·요일” 이 모두 적용되어 있으니,
# 아래처럼 곧장 동별 연간 통계 계산에 쓰실 수 있습니다.



In [9]:
# 3️⃣ 10·20대 방문자 합계 Expr
cols_1020 = ["MALE_10_CNT","MALE_20_CNT","FEML_10_CNT","FEML_20_CNT"]
row_1020 = sum(
    (pl.col(c).cast(pl.Float64) for c in cols_1020),
    pl.lit(0.0)
).alias("ROW_1020")

# 4️⃣ 주말 전체 방문자 합계 Expr (all age columns)
all_age = [
    "MALE_00_CNT","MALE_10_CNT","MALE_20_CNT","MALE_30_CNT",
    "MALE_40_CNT","MALE_50_CNT","MALE_60_CNT","MALE_70_CNT",
    "FEML_00_CNT","FEML_10_CNT","FEML_20_CNT","FEML_30_CNT",
    "FEML_40_CNT","FEML_50_CNT","FEML_60_CNT","FEML_70_CNT"
]
row_total = sum(
    (pl.col(c).cast(pl.Float64) for c in all_age),
    pl.lit(0.0)
).alias("ROW_WEEKEND_TOTAL")

# 5️⃣ ROW 컬럼 추가
step = filtered.with_columns([row_1020, row_total])

# 6️⃣ 동별 연간 합산 & Hotness4 계산
hotness4 = (
    step
      .group_by("FULL_NM")
      .agg([
        pl.sum("ROW_1020").alias("ANNUAL_1020"),
        pl.sum("ROW_WEEKEND_TOTAL").alias("ANNUAL_WEEKEND_TOTAL"),
      ])
      .with_columns([
        (
          pl.col("ANNUAL_1020").sqrt()
          / pl.col("ANNUAL_WEEKEND_TOTAL").sqrt()
        ).alias("HOTNESS4")
      ])
      .sort("HOTNESS4", descending=True)
      .limit(30)
      .collect()
)

print("✅ 동별 연간 Hotness4 TOP30:")
print(hotness4.select(["FULL_NM","HOTNESS4"]))

✅ 동별 연간 Hotness4 TOP30:
shape: (30, 2)
┌───────────────────────────────┬──────────┐
│ FULL_NM                       ┆ HOTNESS4 │
│ ---                           ┆ ---      │
│ str                           ┆ f64      │
╞═══════════════════════════════╪══════════╡
│ 서울특별시_광진구_화양동      ┆ 0.784644 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_서대문구_신촌동    ┆ 0.777264 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_마포구_서교동      ┆ 0.758635 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_성북구_안암동      ┆ 0.742134 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_동대문구_회기동    ┆ 0.730154 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_성동구_사근동      ┆ 0.729808 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 경기도_수원시_팔달구_매산동   ┆ 0.71585  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_관악구_낙성대동    ┆ 0.712107 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 인천광역시_미추홀구_용현1.4동 ┆ 0.701486 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_용산구_이태원1동   ┆ 0.698811

In [10]:
# ── 5) 상위30 평균·중앙값 ──
mean_h4   = hotness4["HOTNESS4"].mean()
median_h4 = hotness4["HOTNESS4"].median()
print(f"\n▶ HOTNESS4 상위30 평균 : {mean_h4:.6f}")
print(f"▶ HOTNESS4 상위30 중앙값: {median_h4:.6f}")


▶ HOTNESS4 상위30 평균 : 0.686935
▶ HOTNESS4 상위30 중앙값: 0.676757


In [14]:
# ─── 난향동(‘서울특별시_관악구_난향동’)의 Hotness4 값 추출 ─────────────────

nanhyang_h4 = (
    hotness4
      .filter(pl.col("FULL_NM") == "서울특별시_관악구_난향동")
      .select(["FULL_NM","HOTNESS4"])
)

print("▶ 난향동 연간 Hotness4:")
print(nanhyang_h4)


▶ 난향동 연간 Hotness4:
shape: (0, 2)
┌─────────┬──────────┐
│ FULL_NM ┆ HOTNESS4 │
│ ---     ┆ ---      │
│ str     ┆ f64      │
╞═════════╪══════════╡
└─────────┴──────────┘


In [12]:
# ─── 1️⃣ Annual4: 난향동 포함 모든 동의 Hotness4 계산 ─────────────────────
annual4 = (
    step
      .group_by("FULL_NM")
      .agg([
        pl.sum("ROW_1020").alias("ANNUAL_1020"),
        pl.sum("ROW_WEEKEND_TOTAL").alias("ANNUAL_WEEKEND_TOTAL"),
      ])
      .with_columns([
        (
          pl.col("ANNUAL_1020").sqrt()
          / pl.col("ANNUAL_WEEKEND_TOTAL").sqrt()
        ).alias("HOTNESS4")
      ])
      .collect()
)

# ─── 2️⃣ Hotness4 > 0 중 최소값 ───────────────────────────────────────
min_positive_h4 = (
    annual4
      .filter(pl.col("HOTNESS4") > 0.0)
      .select(pl.col("HOTNESS4").min())
      .to_series()[0]
)
print(f"▶ Hotness4 > 0 중 최소값: {min_positive_h4:.6f}")


▶ Hotness4 > 0 중 최소값: 0.358208
